In [ ]:
#Author pragati shinde

In [1]:
import pandas as pd 
import numpy as np
import re
import nltk

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_data = pd.read_csv('train.csv', encoding = 'utf-8') 
train_data.head(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0


In [3]:
# 159,571 comments
train_data.nunique()

id               159571
comment_text     159571
toxic                 2
severe_toxic          2
obscene               2
threat                2
insult                2
identity_hate         2
dtype: int64

In [4]:
train_data['toxic'].unique()

array([0, 1])

In [5]:
test_data = pd.read_csv('test.csv', encoding = 'utf-8') 
test_data.head(2)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...


In [6]:
# Predict for 153,164
test_data.nunique()

id              153164
comment_text    153164
dtype: int64

In [7]:
train_data.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [8]:
train_colms = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

In [9]:
#train_data.describe()

In [10]:
# Training data with all ZEROS
unlabelled_comments = train_data[(train_data['toxic']!=1) 
                                 & (train_data['severe_toxic']!=1) 
                                 & (train_data['obscene']!=1) 
                                 & (train_data['threat']!=1) 
                                 & (train_data['insult']!=1) 
                                 & (train_data['identity_hate']!=1)]

print('Percentage of unlabelled comments is ', len(unlabelled_comments)/len(train_data)*100)

# we only have ~10.2% data to 'LEARN' from 

Percentage of unlabelled comments is  89.83211235124176


In [11]:
# No comment is NULL - which is good
null_cmt = train_data[train_data['comment_text'].isnull()]
len(null_cmt)

0

In [12]:
train_data.isnull().any(),test_data.isnull().any()

(id               False
 comment_text     False
 toxic            False
 severe_toxic     False
 obscene          False
 threat           False
 insult           False
 identity_hate    False
 dtype: bool,
 id              False
 comment_text    False
 dtype: bool)

In [13]:
test_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [14]:
# no null comments in test data - good
null_cmt = test_data[test_data['comment_text'].isnull()]
null_cmt

,id,comment_text


In [15]:
y = train_data[train_colms].values

In [16]:
def remove_stop_words(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [17]:
# clean the comment_text in train_data
train_data['comment_text'] = train_data['comment_text'].map(lambda c : remove_stop_words(c))

In [18]:
test_data['comment_text'] = test_data['comment_text'].map(lambda c : remove_stop_words(c))

In [19]:
train_data.head(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i am s...,0,0,0,0,0,0


In [20]:
train_data['comment_text'][6]

'cocksucker before you piss around on my work'

In [21]:
# sample data
train_data[train_data['comment_text'] == str(train_data['comment_text'][6]) ]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,cocksucker before you piss around on my work,1,1,1,0,1,0


In [22]:
test_data.head(2)

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule is more succesful then you wi...
1,0000247867823ef7,from rfc the title is fine as it is imo


In [23]:
# Get only text data for TFIDF
X = train_data.comment_text
test_X = test_data.comment_text

In [24]:
print(X.shape, test_X.shape)

(159571,) (153164,)


In [25]:
159571/(159571+153164) # 51% data for training and 49% on model deployment/testing

0.5102434968903384

In [26]:
X

0         explanation why the edits made under my userna...
1         d aww he matches this background colour i am s...
2         hey man i am really not trying to edit war it ...
3         more i cannot make any real suggestions on imp...
4         you sir are my hero any chance you remember wh...
                                ...                        
159566    and for the second time of asking when your vi...
159567    you should be ashamed of yourself that is a ho...
159568    spitzer umm theres no actual article for prost...
159569    and it looks like it was actually you who put ...
159570    and i really do not think you understand i cam...
Name: comment_text, Length: 159571, dtype: object

In [27]:
test_X

0         yo bitch ja rule is more succesful then you wi...
1                   from rfc the title is fine as it is imo
2                            sources zawe ashton on lapland
3         if you have a look back at the source the info...
4                 i do not anonymously edit articles at all
                                ...                        
153159    i totally agree this stuff is nothing but too ...
153160    throw from out field to home plate does it get...
153161    okinotorishima categories i see your changes a...
153162    one of the founding nations of the eu germany ...
153163    stop already your bullshit is not welcome here...
Name: comment_text, Length: 153164, dtype: object

In [28]:
# import and instantiate TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')
vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [34]:
import joblib
import pickle

In [36]:
# learn the vocabulary in the training data, then use it to create a document-term matrix
# X_dtm = vect.fit_transform(X) -- original

# tf-idf based vectors
#tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), stop_words = "english", lowercase = True, max_features = 500000)

# Fit the model
tf_transformer = vect.fit(X)

# Save FITTED vector for deployment; just TRANSAFORM TEST data (don't FIT TEST data: features will be less)
# Dump the file
pickle.dump(tf_transformer, open("tfidf.pkl", "wb"))

# Fit-Transform
X_dtm = vect.fit_transform(X)

# examine the document-term matrix created from X_train
X_dtm


<159571x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 3178792 stored elements in Compressed Sparse Row format>

In [31]:
test_X.head(2)

0    yo bitch ja rule is more succesful then you wi...
1              from rfc the title is fine as it is imo
Name: comment_text, dtype: object

In [37]:
# transform the test data using the earlier fitted vocabulary, into a document-term matrix
test_X_dtm = vect.transform(test_X)
# examine the document-term matrix from X_test
test_X_dtm

<153164x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2618972 stored elements in Compressed Sparse Row format>

In [38]:
train_colms = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

In [39]:
# STrategy: We have 6 classes to predict 
# so we will build 6 seperate models and combine result later

In [40]:
# import and instantiate the Logistic Regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=12.0)

# create submission file
submission_binary = pd.read_csv('sample_submission.csv')

for label in train_colms:
    print('... Processing {}'.format(label))
    y = train_data[label]
    # train the model using X_dtm & y
    logreg.fit(X_dtm, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(test_X_dtm)[:,1]
    submission_binary[label] = test_y_prob
    

... Processing obscene
Training accuracy is 0.9832488359413678
... Processing insult
Training accuracy is 0.9755344016143285
... Processing toxic
Training accuracy is 0.9639533499194716
... Processing severe_toxic
Training accuracy is 0.9920599607698141
... Processing identity_hate
Training accuracy is 0.9939838692494251
... Processing threat
Training accuracy is 0.9981199591404453


In [41]:
type(submission_binary)

pandas.core.frame.DataFrame

In [42]:
submission_binary.head()

,id,obscene,insult,toxic,severe_toxic,identity_hate,threat
0,00001cee341fdb12,0.999965,0.972258,0.999958,0.463250,0.440776,0.050124
1,0000247867823ef7,0.000401,0.003252,0.002461,0.000442,0.000405,0.000324
2,00013b17ad220c46,0.003198,0.007652,0.010813,0.000116,0.001212,0.000035
3,00017563c3f7919a,0.000975,0.000835,0.001347,0.002146,0.000032,0.000117
4,00017695ad8997eb,0.001241,0.003271,0.019376,0.000806,0.000558,0.000602


In [43]:
test_data.head(2)

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule is more succesful then you wi...
1,0000247867823ef7,from rfc the title is fine as it is imo


In [44]:
submission_binary.to_csv('final_preds.csv', encoding = 'utf-8')

In [ ]:
# Model dump: After Training 